In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications.densenet import DenseNet169

from keras.models import Sequential
from tensorflow.keras import regularizers as reg
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout, SpatialDropout2D, BatchNormalization, Resizing, Rescaling, RandomFlip, RandomRotation

In [2]:
# finue tune transfer learning model
dense_net = DenseNet169(weights='imagenet', include_top=False, input_shape=None)

for layer in dense_net.layers:
    layer.trainable = False

In [5]:
# Images Dirs 
batch_size = 32
train_dir = 'data/train'
test_dir = 'data/test'

In [6]:
# Getting train , test , validate Images from our dirs 
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(200, 200),
    batch_size=batch_size)

valid_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(200, 200),
    batch_size=batch_size)

Found 274 files belonging to 5 classes.
Found 180 files belonging to 5 classes.


In [7]:
# Build Model

model = Sequential()
model.add(dense_net)
model.add(GlobalAveragePooling2D())
model.add(Dense(256 , activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(128 , activation ='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, None, None, 1664   12642880  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 1664)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               426240    
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                        

In [8]:
model_history = model.fit(train_data, epochs=20, validation_data=valid_data, verbose=1)

Epoch 1/20
9/9 [==============================] - 16s 999ms/step - loss: 1.7143 - accuracy: 0.3321 - val_loss: 3.6362 - val_accuracy: 0.3667
Epoch 2/20
9/9 [==============================] - 3s 277ms/step - loss: 0.9609 - accuracy: 0.6241 - val_loss: 0.7148 - val_accuracy: 0.8389
Epoch 3/20
9/9 [==============================] - 2s 243ms/step - loss: 0.6996 - accuracy: 0.7847 - val_loss: 0.9618 - val_accuracy: 0.7167
Epoch 4/20
9/9 [==============================] - 2s 253ms/step - loss: 0.5872 - accuracy: 0.8102 - val_loss: 0.4208 - val_accuracy: 0.8833
Epoch 5/20
9/9 [==============================] - 2s 211ms/step - loss: 0.4668 - accuracy: 0.8467 - val_loss: 0.4297 - val_accuracy: 0.8722
Epoch 6/20
9/9 [==============================] - 2s 231ms/step - loss: 0.3703 - accuracy: 0.9307 - val_loss: 0.5450 - val_accuracy: 0.7944
Epoch 7/20
9/9 [==============================] - 2s 227ms/step - loss: 0.2949 - accuracy: 0.9453 - val_loss: 0.2135 - val_accuracy: 0.9333
Epoch 8/20
9/9 [===

In [9]:
model.save('cnn_dense.h5')

/Users/mahmoudahmed/opt/anaconda3/envs/deeplearning/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np

from keras.preprocessing import image
model = load_model('cnn_dense.h5')


model_dict = {0: "chris_evans", 1: "chris_hemsworth", 2: "mark_ruffalo", 3: "robert_downey_jr", 4: "scarlett_johansson"}

# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face


# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture("test.mp4")

while True:
    _, frame = video_capture.read()    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (180, 180))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)[0]
        maxindex = int(np.argmax(pred))
        print(maxindex)
                     
        name="None matching"
        print(pred[maxindex])
        if pred[maxindex] > .4:
            name = model_dict[maxindex]
#         if(pred[0][3]>0.3):
            
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()